# Práctica 3 - Diffie Hellman

- Diego García Díaz
- Alberto Pérez Álvarez

## Generación de claves con Diffie Hellman

In [2]:
import random, math

def es_primo(numero):
  """
  Verifica si un número entero es primo.
  True si el número es primo, False de lo contrario.
  """
  if not isinstance(numero, int):
      raise TypeError("El input debe ser un número entero.")
  if numero <= 1:
    return False
  if numero <= 3:
      return True # 2 y 3 son primos
  if numero % 2 == 0 or numero % 3 == 0:
      return False # Eliminar múltiplos de 2 y 3
  # Solo necesitamos comprobar divisores de la forma 6k ± 1 hasta sqrt(n)
  i = 5
  while i * i <= numero:
    if numero % i == 0 or numero % (i + 2) == 0:
      return False
    i += 6
  return True

def genera_un_primo_aleatorio(max_val, min_val=1, intentos_max=10000):
    """
    Función auxiliar para generar un único primo aleatorio en el rango [min_val, max_val).
    """
    if min_val >= max_val:
        raise ValueError("El valor mínimo debe ser estrictamente menor que el máximo.")

    for _ in range(intentos_max):
        candidato = random.randrange(min_val, max_val)
        if es_primo(candidato):
            return candidato

    # Si llegamos aquí, no se encontró un primo en los intentos dados
    raise ValueError(f"No se pudo encontrar un primo en el rango [{min_val}, {max_val}) "
                     f"después de {intentos_max} intentos.")


El codigo de arriba permite crear un numero primo aleatorio dado un limite superior, y se puede modificar el inferior y los intentos, en caso de que el numero a generar sea muy grande.

In [3]:
def exponencia_modular(base, expo, mod):
    if not isinstance(base, int) or not isinstance(expo, int) or not isinstance(mod, int):
        raise TypeError("Las entradas deben ser int")
    if expo < 0:
        raise ValueError("El exponente no puede ser negativo para este algoritmo")
    if mod <= 0:
        raise ValueError("El módulo debe ser un positivo")

    i = expo  
    x = base % mod 
    r = 1  

    while i > 0:
        if i % 2 != 0:
            r = (r * x) % mod 
            
        x = (x * x) % mod
        i = i // 2 

    return r

Implementacion de la version iterativa mejorada de las diapositivas (diapo 37)

In [4]:
def diffie_hellman():
    p_primo = genera_un_primo_aleatorio(1000000000000)

    n_primo = genera_un_primo_aleatorio(p_primo)
    print(f"El valor de p = {p_primo} \nEl valor de la base = {n_primo}")

    random.seed(55)

    clave_privada_Alice = random.randrange(1,p_primo)
    

    clave_publica_Alice = exponencia_modular(n_primo, clave_privada_Alice, p_primo)
    print(f"\nEl mensaje de Alice a Bob es {clave_publica_Alice}")

    clave_privada_Bob = random.randrange(1,p_primo)

    clave_publica_Bob = exponencia_modular(n_primo, clave_privada_Bob, p_primo)
    print(f"El mensaje de Bob a Alice es {clave_publica_Bob}")

    print("\nLa clave privada de Alice es",clave_privada_Alice)
    print("La clave privada de Bob es",clave_privada_Bob)

    clave_mensaje_Alice = exponencia_modular(clave_publica_Bob, clave_privada_Alice, p_primo)

    clave_mensaje_Bob = exponencia_modular(clave_publica_Alice, clave_privada_Bob, p_primo)

    print(f"\nLas clave del mensaje en ambos lados es igual == {clave_mensaje_Bob == clave_mensaje_Alice}")
    print(f"La clave == {clave_mensaje_Alice} = {clave_mensaje_Bob}")

    return (clave_publica_Alice,clave_publica_Bob)
    



Implementación del procedimiento Diffie-Hellman. Ejemplo abajo:

In [5]:
diffie_hellman()

El valor de p = 848551374799 
El valor de la base = 260740838723

El mensaje de Alice a Bob es 210553389852
El mensaje de Bob a Alice es 722520971632

La clave privada de Alice es 218831304874
La clave privada de Bob es 333888053372

Las clave del mensaje en ambos lados es igual == True
La clave == 848362756228 = 848362756228


(210553389852, 722520971632)

In [6]:
def generar_numero_de_n_digitos(n):
  """Genera un numero de N digitos y lo devuelve
  """
  if not isinstance(n, int) or n <= 0:
    print("Error: n tiene que ser positivo.")
    return None

  if n == 1:
    return str(random.randint(0, 9))  # Pequeño

  # Generar primer digito
  primer_digito = str(random.randint(1, 9)) # No puede empezar por 0

  # Generar el resto de digitos
  tolresto_digitos = ''.join(str(random.randint(0, 9)) for _ in range(n - 1))

  return int(primer_digito + tolresto_digitos)

In [7]:
def diffie_hellman_DHGroup_18():
    random.seed(55)
    '''
    Algoritmo de Diffie Hellman pero usando el grupo 18 para una conexión segura
    Definido en https://datatracker.ietf.org/doc/html/rfc3526#section-7 por el IETF
    
    p = 2^8192 - 2^8128 - 1 + 2^64 * { [2^8062 pi] + 4743158 }
    g = 2
    Las claves privadas deben tener una longitud de 8192*log10(2) = 2466 digitos
    '''
    # 2^8192 - 2^8128 - 1 + 2^64 * { [2^8062 pi] + 4743158 }
    p = 4260734904763343474415214567811658204484980956848328952242582449086524097354631402455204975972018819412938141285454816286033979502635459620078102600724364193734988460823915382971476955108832160341063796187431387810451953905706920362236486440611541026827811510847592810122953074775473890151248349684175046214374685261484212840185706897329762372919919242797759881543785829207036631215813076421026877183051106746774039120996620676269984436626245665218596476607867803598759873201050064528591145849621542098407657860475472730296228037560470579997081050262970165567795299225762198463891475443671171369665318601202395527944156057794028252548456621030740203806568954311732417505336759799403373704595758324340243503271441405374000182553327450153888099766985995597088436403494899700977292516366494314611903230419110002707752869745866823971275688008831922630001238951939136607632633443149186558959841894340299631161570599536403500160942513734977613454645537998829256578591641732345516821109882909819843676312140730405735179567649093045628324213475368816046690284785009394288186678511395341235762607246352280530654957560666209982186794845006769241676051313070645806180994643536939054967199445514196130416830872291194934373470796189506756348190074719613675655216603593564079094524464116527724118209571796942491863184859901744432348856785224143868569064934438816208872289811275149411350337552266527523814261537476911623204238426923714567949664173068544736660008102771242780568116808500050358395918744837580483289198829341269113462874067985679447992734522843870766767747162830209531377490777737400399888837809775287884330831611019322183698852805452742997604257682648197012071519110598518880335545183865879569985508518348726810008097876862483307652319596792352081624928802898869497890053730393205770216565027643520610283979368258045719211723681190240281960838994269299412919439862430051258623556060988056569037208411097803033625563526564039329160951026895687092143055400154641292662431613209789820102520638326905985417632070782127846892165509726958647618927935441267301112383277679138532690431054976285478645967700843895050720077657002098327272896719602457783580669541927773743618311320691973016680091056472620692407741265010630436069785112102788657925640022376954668847225631841209904635085874626893125004097660890355212716528901396634945423116475654633619293205602958310596962441679333882638350769356026859679445374334434787163400202054177220448504981127301099825113836355059711
    print("El modulo es",p)

    g = 2

    clave_privada_Alice = generar_numero_de_n_digitos(2466)

    clave_publica_Alice = exponencia_modular(g, clave_privada_Alice, p)
    print(f"\nEl mensaje de Alice a Bob es {clave_publica_Alice}")

    clave_privada_Bob = generar_numero_de_n_digitos(2466)

    clave_publica_Bob = exponencia_modular(g, clave_privada_Bob, p)
    print(f"El mensaje de Bob a Alice es {clave_publica_Bob}")

    print("\nLa clave privada de Alice es",clave_privada_Alice)
    print("La clave privada de Bob es",clave_privada_Bob)

    clave_mensaje_Alice = exponencia_modular(clave_publica_Bob, clave_privada_Alice, p)

    clave_mensaje_Bob = exponencia_modular(clave_publica_Alice, clave_privada_Bob, p)

    print(f"\nLas clave del mensaje en ambos lados es igual --> {clave_mensaje_Bob == clave_mensaje_Alice}")
    print(f"La clave de Alice ({clave_mensaje_Alice}) es igual a la de Bob ({clave_mensaje_Bob})")
    return (clave_publica_Alice,clave_publica_Bob)
    

In [8]:
diffie_hellman_DHGroup_18()

El modulo es 426073490476334347441521456781165820448498095684832895224258244908652409735463140245520497597201881941293814128545481628603397950263545962007810260072436419373498846082391538297147695510883216034106379618743138781045195390570692036223648644061154102682781151084759281012295307477547389015124834968417504621437468526148421284018570689732976237291991924279775988154378582920703663121581307642102687718305110674677403912099662067626998443662624566521859647660786780359875987320105006452859114584962154209840765786047547273029622803756047057999708105026297016556779529922576219846389147544367117136966531860120239552794415605779402825254845662103074020380656895431173241750533675979940337370459575832434024350327144140537400018255332745015388809976698599559708843640349489970097729251636649431461190323041911000270775286974586682397127568800883192263000123895193913660763263344314918655895984189434029963116157059953640350016094251373497761345464553799882925657859164173234551682110988290981

(243956306199637292688859890560354574468130522256327378800059490757927061466114006537335444001457099283955529564094745521594344617286028141806580248398644761504119061421914884152743758534150423967244830085581220274392067767550264892999641417928865572478745658347822385511556447652181318933833327301165876981782568444306078273887977128874057659237897961455799219554332335684007909136346089052463214475124895476116459409432612428984682547187307246427114065844160038853448308105675725351665232647114666980276725134582328624698611676375754698132655392560150022507342056066879834451841148904081413333266019684631975598647086961473450312351844435864892810038844657901673428271668139126314019374469324154875307280093411126511627706120408991979282057216545291713070359717858349396590612977501204875120420929562353962480362471447224067837920783906892865172025837723321169537227811869835866856860274132746721016158009182785965030103041576066054216026773636976729384134593439251198598287438574595988656521838499

## Fuerza Bruta

In [9]:
def mostrarTiempo(tiempo):  # Para imprimir los tiempos con un formato bonito
    horas = int(tiempo // 3600)
    minutos = int((tiempo % 3600) // 60)
    segundos = int(tiempo % 60)
    ms = int((tiempo - int(tiempo)) * 1000000)
    return f"{horas:01d}:{minutos:02d}:{segundos:02d}.{ms:06d}"

In [10]:
import time

def fuerza_bruta(g,p,mensaje_Alice,mensaje_Bob):
    t_inicio = time.time()
    exponente = 0
    resultado = -1
    soluciones = []
    while len(soluciones) < 2:
        exponente += 1
        resultado = exponencia_modular(g,exponente,p)
        if resultado == mensaje_Alice:
            soluciones.append(("Alice",exponente,time.time()-t_inicio))

        elif resultado == mensaje_Bob:
            soluciones.append(("Bob",exponente,time.time()-t_inicio))
    
    if soluciones[0][0] == "Alice":
        expo = soluciones[0][1]
        clave_uno = exponencia_modular(mensaje_Bob,expo,p)

        expo = soluciones[1][1]
        clave_dos = exponencia_modular(mensaje_Alice,expo,p)

        if clave_uno == clave_dos:
            soluciones.append(("Clave",clave_uno,time.time()-t_inicio))
    elif soluciones[0][0] == "Bob":
        expo = soluciones[0][1]
        clave_uno = exponencia_modular(mensaje_Alice,expo,p)

        expo = soluciones[1][1]
        clave_dos = exponencia_modular(mensaje_Bob,expo,p)

        if clave_uno == clave_dos:
            soluciones.append(("Clave",clave_uno,time.time()-t_inicio))

    return soluciones

### Fuerza bruta paralela (NO FUNCIONA EN EL NOTEBOOK, funciona en el archivo adjunto)

In [11]:
import time
import multiprocessing
import math

def worker_fuerza_bruta(args):
    """
    Función ejecutada por cada proceso trabajador.
    Verifica un solo exponente.
    (Asegúrate de que esté aquí, fuera del if __name__ == "__main__":)
    """
    g, p, mensaje_Alice, mensaje_Bob, exponente = args
    try:
        resultado = exponencia_modular(g, exponente, p)
        if resultado == mensaje_Alice:
            return ("Alice", exponente)
        elif resultado == mensaje_Bob:
            return ("Bob", exponente)
        else:
            return None
    except Exception as e:
        print(f"Error en worker con exponente {exponente}: {e}") # Descomentar para depurar errores en workers
        return None

def genera_argumentos(g, p, mensaje_Alice, mensaje_Bob, inicio=1):
    """
    Generador infinito que produce los argumentos para cada llamada
    a worker_fuerza_bruta, incrementando el exponente.
    (Asegúrate de que esté aquí, fuera del if __name__ == "__main__":)
    """
    exponente = inicio - 1
    while True:
        exponente += 1
        yield (g, p, mensaje_Alice, mensaje_Bob, exponente)


def fuerza_bruta_paralela(g, p, mensaje_Alice, mensaje_Bob, num_procesos=None):
    """
    Realiza la búsqueda de fuerza bruta en paralelo utilizando múltiples procesos.

    num_procesos: Número de procesos a utilizar. Si es None, usa todos los cores disponibles.
    """
    if num_procesos is None:
        try:
            num_procesos = multiprocessing.cpu_count()
            print(f"Detectados {num_procesos} núcleos de CPU. Usando {num_procesos} procesos.")
        except NotImplementedError:
            print("No se pudo detectar el número de núcleos. Usando 2 procesos por defecto.")
            num_procesos = 2 # Usar un valor por defecto si cpu_count falla
    else:
        print(f"Usando {num_procesos} procesos especificados.")

    t_inicio = time.time()
    soluciones_encontradas = {} # Diccionario para almacenar las soluciones {Nombre: (exponente, tiempo)}
    soluciones_final = []

    # Usamos un Pool de procesos
    # with asegura que los procesos se cierren correctamente
    with multiprocessing.Pool(processes=num_procesos) as pool:
        print(f"Iniciando búsqueda paralela para g={g}, p={p}, A={mensaje_Alice}, B={mensaje_Bob}")

        # chunksize puede ayudar a mejorar el rendimiento al enviar trabajo en lotes
        chunksize = 1000

        # pool.imap_unordered procesa las tareas y devuelve los resultados
        # tan pronto como están listos, sin mantener el orden original.
        # Esto es ideal porque queremos encontrar las claves lo antes posible.
        resultados_iter = pool.imap_unordered(
            worker_fuerza_bruta,
            genera_argumentos(g, p, mensaje_Alice, mensaje_Bob),
            chunksize=chunksize
        )

        try:
            for resultado in resultados_iter:
                if resultado:  # Si el worker devolvió algo no None
                    quien, exponente = resultado
                    if quien not in soluciones_encontradas: # Solo registrar la primera vez que se encuentra
                        tiempo_transcurrido = time.time() - t_inicio
                        soluciones_encontradas[quien] = (exponente, tiempo_transcurrido)
                        print(f"¡Encontrado! Clave de {quien}: {exponente} (Tiempo: {mostrarTiempo(tiempo_transcurrido)} segundos)")

                        # Si ya hemos encontrado ambas claves, podemos detener la búsqueda
                        if len(soluciones_encontradas) == 2:
                            print("Ambas claves encontradas. Terminando procesos...")
                            pool.terminate()
                            # pool.join() espera a que terminen (importante después de terminate)
                            pool.join()
                            break 

        except KeyboardInterrupt:
            print("\nBúsqueda interrumpida por el usuario.")
            pool.terminate()
            pool.join()
        except Exception as e:
            print(f"\nOcurrió un error durante la búsqueda: {e}")
            pool.terminate()
            pool.join()


    if "Alice" in soluciones_encontradas:
        exp, t = soluciones_encontradas["Alice"]
        soluciones_final.append(("Alice", exp, t))
    if "Bob" in soluciones_encontradas:
        exp, t = soluciones_encontradas["Bob"]
        soluciones_final.append(("Bob", exp, t))


    t_total = time.time() - t_inicio
    print(f"Búsqueda completada (o detenida). Tiempo total: {mostrarTiempo(t_total)} segundos.")
    return soluciones_final


In [16]:
'''
Ejemplo pequeño
'''
g = 4
p = 19
clave_Alice = 8
clave_Bob = 9
publicacion_Alice = exponencia_modular(g,clave_Alice,p)
publicacion_Bob = exponencia_modular(g,clave_Bob,p)
soluciones = fuerza_bruta(g,p,publicacion_Alice,publicacion_Bob)
print("¡Se han encontrado las claves privadas!")
print(f"Primera clave obtenida: \n   De {soluciones[0][0]}\n   Clave: {soluciones[0][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[0][2])}")
print(f"Segunda clave obtenida: \n   De {soluciones[1][0]}\n   Clave: {soluciones[1][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[1][2])}")
print(f"La clave privada común: \n   {soluciones[2][0]}: {soluciones[2][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[2][2])}")




¡Se han encontrado las claves privadas!
Primera clave obtenida: 
   De Alice
   Clave: 8
   Tiempo empleado: 0:00:00.000006
Segunda clave obtenida: 
   De Bob
   Clave: 9
   Tiempo empleado: 0:00:00.000008
La clave privada común: 
   Clave: 1
   Tiempo empleado: 0:00:00.000012


In [17]:
'''
FUERZA BRUTA PARA ENCONTRAR LA CLAVE DE LOS PRIMEROS DATOS DEL PDF DE LA PRÁCTICA
'''

g = 12345701
p = 666666653
publicacion_Alice = 518633585
publicacion_Bob = 654181592
print("\nBuscando claves privadas...")
soluciones = fuerza_bruta(g,p,publicacion_Alice,publicacion_Bob)
print("¡Se han encontrado las claves privadas!")
print(f"Primera clave obtenida: \n   De {soluciones[0][0]}\n   Clave: {soluciones[0][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[0][2])}")
print(f"Segunda clave obtenida: \n   De {soluciones[1][0]}\n   Clave: {soluciones[1][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[1][2])}")
print(f"La clave privada común: \n   {soluciones[2][0]}: {soluciones[2][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[2][2])}")


Buscando claves privadas...
¡Se han encontrado las claves privadas!
Primera clave obtenida: 
   De Bob
   Clave: 271828182
   Tiempo empleado: 0:13:02.073791
Segunda clave obtenida: 
   De Alice
   Clave: 314159265
   Tiempo empleado: 0:15:07.730872
La clave privada común: 
   Clave: 410688564
   Tiempo empleado: 0:15:07.730886


In [ ]:
'''
Valores del grupo 18
'''
g = 2
p = 4260734904763343474415214567811658204484980956848328952242582449086524097354631402455204975972018819412938141285454816286033979502635459620078102600724364193734988460823915382971476955108832160341063796187431387810451953905706920362236486440611541026827811510847592810122953074775473890151248349684175046214374685261484212840185706897329762372919919242797759881543785829207036631215813076421026877183051106746774039120996620676269984436626245665218596476607867803598759873201050064528591145849621542098407657860475472730296228037560470579997081050262970165567795299225762198463891475443671171369665318601202395527944156057794028252548456621030740203806568954311732417505336759799403373704595758324340243503271441405374000182553327450153888099766985995597088436403494899700977292516366494314611903230419110002707752869745866823971275688008831922630001238951939136607632633443149186558959841894340299631161570599536403500160942513734977613454645537998829256578591641732345516821109882909819843676312140730405735179567649093045628324213475368816046690284785009394288186678511395341235762607246352280530654957560666209982186794845006769241676051313070645806180994643536939054967199445514196130416830872291194934373470796189506756348190074719613675655216603593564079094524464116527724118209571796942491863184859901744432348856785224143868569064934438816208872289811275149411350337552266527523814261537476911623204238426923714567949664173068544736660008102771242780568116808500050358395918744837580483289198829341269113462874067985679447992734522843870766767747162830209531377490777737400399888837809775287884330831611019322183698852805452742997604257682648197012071519110598518880335545183865879569985508518348726810008097876862483307652319596792352081624928802898869497890053730393205770216565027643520610283979368258045719211723681190240281960838994269299412919439862430051258623556060988056569037208411097803033625563526564039329160951026895687092143055400154641292662431613209789820102520638326905985417632070782127846892165509726958647618927935441267301112383277679138532690431054976285478645967700843895050720077657002098327272896719602457783580669541927773743618311320691973016680091056472620692407741265010630436069785112102788657925640022376954668847225631841209904635085874626893125004097660890355212716528901396634945423116475654633619293205602958310596962441679333882638350769356026859679445374334434787163400202054177220448504981127301099825113836355059711
dh = diffie_hellman_DHGroup_18()
print("\nBuscando claves privadas...")
publicacion_Alice = dh[0]
publicacion_Bob = dh[1]
soluciones = fuerza_bruta(g,p,publicacion_Alice,publicacion_Bob)
print("¡Se han encontrado las claves privadas!")
print(f"Primera clave obtenida: \n   De {soluciones[0][0]}\n   Clave: {soluciones[0][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[0][2])}")
print(f"Segunda clave obtenida: \n   De {soluciones[1][0]}\n   Clave: {soluciones[1][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[1][2])}")

El modulo es 426073490476334347441521456781165820448498095684832895224258244908652409735463140245520497597201881941293814128545481628603397950263545962007810260072436419373498846082391538297147695510883216034106379618743138781045195390570692036223648644061154102682781151084759281012295307477547389015124834968417504621437468526148421284018570689732976237291991924279775988154378582920703663121581307642102687718305110674677403912099662067626998443662624566521859647660786780359875987320105006452859114584962154209840765786047547273029622803756047057999708105026297016556779529922576219846389147544367117136966531860120239552794415605779402825254845662103074020380656895431173241750533675979940337370459575832434024350327144140537400018255332745015388809976698599559708843640349489970097729251636649431461190323041911000270775286974586682397127568800883192263000123895193913660763263344314918655895984189434029963116157059953640350016094251373497761345464553799882925657859164173234551682110988290981

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/resource_tracker.py:104: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/resource_tracker.py", line 209, in main
    cache[rtype].remove(name)
KeyError: '/mp-kgh4ylm6'
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/resource_tracker.py", line 209, in main
    cache[rtype].remove(name)
KeyError: '/mp-y8ul0evc'
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/resource_tracker.py", line 209, in main
    cache[rtype].remove(name)
KeyError: '/mp-m8ekdiz1'
Traceback (most recent call last):
  File "/Library/Frameworks/Python.

KeyboardInterrupt: 

In [ ]:
'''
PRIMER EJEMPLO DE LA PRÁCTICA
'''

g = 7862847011909
p = 9460397478313
publicacion_Alice = 6517745061690
publicacion_Bob = 2546007328599
print("\nBuscando claves privadas...")
soluciones = fuerza_bruta(g,p,publicacion_Alice,publicacion_Bob)
print("¡Se han encontrado las claves privadas!")
print(f"Primera clave obtenida: \n   De {soluciones[0][0]}\n   Clave: {soluciones[0][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[0][2])}")
print(f"Segunda clave obtenida: \n   De {soluciones[1][0]}\n   Clave: {soluciones[1][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[1][2])}")
print(f"La clave privada común: \n   {soluciones[2][0]}: {soluciones[2][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[2][2])}")

El valor de p = 711491147 
El valor de la base = 506765719

El mensaje de Alice a Bob es 118334279
El mensaje de Bob a Alice es 213545450

La clave privada de Alice es 97001677
La clave privada de Bob es 210718381

Las clave del mensaje en ambos lados es igual == True
La clave == 492447201 = 492447201

Buscando claves privadas...
¡Se han encontrado las claves privadas!
Primera clave obtenida: 
   De Alice
   Clave: 97001677
   Tiempo empleado: 0:04:28.758059
Segunda clave obtenida: 
   De Bob
   Clave: 210718381
   Tiempo empleado: 0:10:15.949985
